In [ ]:
!pip install --upgrade pip
!pip3 install tensorflow
#!pip3 install tensorflow==2.4.0
!pip install gym
!pip install keras
!pip install keras-rl2

In [ ]:
!pip3 install tk

In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
from tkinter import *
import math

In [14]:
class ShowerEnv(Env):
    def __init__(self):
        #self.produced_energy = float(input("Energía a producir: "))
        
        #Acctions up, down stay
        self.action_space = Discrete(3)
        #Angle array
        self.observation_space = Box(low=np.array([0]), high=np.array([180]))
        #Set start angle
        self.state = random.uniform(0, 0.593)
        #Set reaching energy producted
        self.training_length = 60
        
        win = Tk()
        
        e = Entry(win)
        e.pack()
        e.insert(0, "Energía")
        win.geometry("200x100")
        
        def myClick():
            global eficiency
            energy = float(e.get())
            eficiency = energy/((1/2)*1.225*math.pi*pow(2, 2)*pow(10, 3))
            win.quit()
            win.withdraw()
            
        myButton = Button(win, text="Iniciar", command = myClick)
        myButton.pack()
        
        win.mainloop()
        
    def step(self, action):
        #Apply action
        #si la accion es 0, le resta 1 = -1 (temp disminuye 1 grado)
        #si la accion es 1, le resta 1 = 0 (temp ni aumenta ni disminuye)
        #si la accion es 2, le resta 1 = 1 grados (temp aumenta 1 grado)
        self.last_state = state
        self.state += action - 1
        #Reduce el tiempo de ducha en 1 segundo(el total de duracion es 60)
        self.training_length -= 1
        
        #Calcula el reward
        if abs(self.state-eficiency) < abs(self.last_state-eficiency):
            reward = 1
        elif abs(self.state-eficiency) == abs(self.last_state-eficiency):
            reward = -1
        else:
            reward = -10
        
        #Checkea si la ducha ha terminado
        if self.training_length <=0:
            done = True
        else:
            done = False
            
        #Perturbaciones en el viento
        self.state += random.randint(-10,10)
        #placeholder para la info
        info = {}
        
        #Retorna informacion del paso(step information)
        return self.state, reward, done, info
    
    def render(self):
        #visualización
        win = Tk()
        
        win.geometry("750x250")
        def callback():
            Label(win, text="Hello World!", font=('Century 20 bold')).pack(pady=4)
        
        win.bind('<Return>',lambda event:callback())
        #win.mainloop()
        
    def reset(self):
        #Reseteamos el ángulo
        self.state = random.randint(0,180)
        #Reseteamos el tiempo de entreamiento
        self.training_length = 60
        return self.state

In [15]:
env= ShowerEnv()

In [16]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))


Episode:1 Score:-547
Episode:2 Score:-281
Episode:3 Score:-483
Episode:4 Score:-448
Episode:5 Score:-600
Episode:6 Score:-591
Episode:7 Score:34
Episode:8 Score:49
Episode:9 Score:-558
Episode:10 Score:-446


In [17]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [18]:
states = env.observation_space.shape
actions = env.action_space.n

In [19]:
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(24, activation='relu', input_shape = states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [25]:
del model

In [26]:
model = build_model(states, actions)

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                48        
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [22]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [27]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [28]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 11:30 - reward: 1.0000

C:\Users\dgarr\anaconda3\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
C:\Users\dgarr\anaconda3\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 64s 6ms/step - reward: 0.0581
166 episodes - episode_reward: 3.259 [-600.000, 60.000] - loss: 34.328 - mae: 25.462 - mean_q: 5.306

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 65s 6ms/step - reward: -0.1476
167 episodes - episode_reward: -8.126 [-600.000, 60.000] - loss: 59.094 - mae: 33.180 - mean_q: 20.484

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 66s 7ms/step - reward: 0.2448
167 episodes - episode_reward: 14.186 [-600.000, 60.000] - loss: 72.632 - mae: 34.090 - mean_q: 20.827

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 68s 7ms/step - reward: 0.2294
166 episodes - episode_reward: 13.578 [-600.000, 60.000] - loss: 70.237 - mae: 32.536 - mean_q: 22.827

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 69s 7ms/step - reward: 0.3872
done, took 331.070 seconds
